In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action = 'ignore')
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)

In [2]:
vod_08 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_VOD_2308.csv', encoding = 'cp949', sep = '\t')
vod_09 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_VOD_2309.csv', encoding = 'cp949', sep = '\t')

In [97]:
# 8,9월 데이터 합치기
vod_89 = pd.concat([vod_08, vod_09], ignore_index=True)
vod_89.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227
2,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230829194727
3,61689000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회(22/07/21),TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",1:13,20230804092737


In [15]:
from pandas._libs.tslibs.offsets import Hour
def preprocessing(df):
  # 특수문자 정리
  ## 예고편 제거
  ## 괄호와 그 안의 내용 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\(.*$', '', regex=True)

  # use_tms 분 단위로 변경
  df['use_tms'] = round(df['use_tms'] / 60, 1)

  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  return df

In [16]:
df = preprocessing(vod_89)
df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737


In [25]:
df = df.dropna()

In [26]:
df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737


In [35]:
# 프로그램 회차별 정보
df1 = df[['asset_nm', 'ct_cl', 'genre_of_ct_cl', 'ACTR_DISP', 'disp_rtm']].drop_duplicates().reset_index(drop = True)
df1.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78


In [36]:
# 유저마다 각 회차를 본 시간을 더함
df2 = pd.DataFrame(df.groupby(['subsr','asset_nm'])['use_tms'].sum()).reset_index()
df2.head()

,subsr,asset_nm,use_tms
0,59879000,소방서 옆 경찰서 05회,2.4
1,59879000,소방서 옆 경찰서 06회,22.1
2,59879000,신성한 이혼 01회,12.6
3,59879000,신성한 이혼 02회,16.8
4,59879000,신성한 이혼 03회,4.3


In [37]:
# 회차별 정보와 유저 시청정보를 합친 데이터프레임
df3 = df2.merge(df1, on = 'asset_nm', how = 'left')
df3 = df3[(df3['disp_rtm'] != 0) & (df3['use_tms'] != 0)]
df3.head()

,subsr,asset_nm,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,59879000,소방서 옆 경찰서 05회,2.4,TV드라마,기타,"김래원,손호준,공승연",69
1,59879000,소방서 옆 경찰서 06회,22.1,TV드라마,기타,"김래원,손호준,공승연",59
2,59879000,신성한 이혼 01회,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64
3,59879000,신성한 이혼 02회,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62
4,59879000,신성한 이혼 03회,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63


In [39]:
# 시청시간으로 유저가 이 회차를 시청했는지 여부를 결정
## 시청시간을 런타임으로 나눠 0.1 이 넘으면 시청한 것으로 간주
df3['watched'] = df3['use_tms'] / df3['disp_rtm']
df3['watched'] = df3['watched'].apply(lambda x : 1 if x >= 0.1 else 0)
df3.head()

,subsr,asset_nm,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,watched
0,59879000,소방서 옆 경찰서 05회,2.4,TV드라마,기타,"김래원,손호준,공승연",69,0
1,59879000,소방서 옆 경찰서 06회,22.1,TV드라마,기타,"김래원,손호준,공승연",59,1
2,59879000,신성한 이혼 01회,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,1
3,59879000,신성한 이혼 02회,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62,1
4,59879000,신성한 이혼 03회,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63,0


In [40]:
df1

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78
...,...,...,...,...,...
4140,엄마가 화났다,키즈,학습,-,8
4141,핑크퐁 자동차 동화 01회,키즈,학습,핑크퐁,4
4142,간질간질,키즈,학습,-,6
4143,월간 아기상어 01회,키즈,학습,아기상어,1


In [41]:
# 회차 제거
df11 = df1.copy()

df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
df11.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",78


In [53]:
# 유저 시청 정보의 회차 제거한 변수 추가
df3['series_nm'] = df3['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df3['series_nm'] = df3['series_nm'].str.replace(r'\d+회\.', '', regex=True)
df3.head()

,subsr,asset_nm,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,2.4,TV드라마,기타,"김래원,손호준,공승연",69,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,22.1,TV드라마,기타,"김래원,손호준,공승연",59,1,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,1,신성한 이혼
3,59879000,신성한 이혼 02회,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62,1,신성한 이혼
4,59879000,신성한 이혼 03회,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63,0,신성한 이혼


In [54]:
df33 = df3[['subsr', 'series_nm', 'ct_cl',	'genre_of_ct_cl', 'ACTR_DISP',	'watched']].copy()
df33.head()

,subsr,series_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,watched
0,59879000,소방서 옆 경찰서,TV드라마,기타,"김래원,손호준,공승연",0
1,59879000,소방서 옆 경찰서,TV드라마,기타,"김래원,손호준,공승연",1
2,59879000,신성한 이혼,TV드라마,기타,"조승우,한혜진,김성균,정문성",1
3,59879000,신성한 이혼,TV드라마,기타,"조승우,한혜진,김성균,정문성",1
4,59879000,신성한 이혼,TV드라마,기타,"조승우,한혜진,김성균,정문성",0


In [59]:
# 유저의 시리즈별 시청 횟수
df4 = df33.groupby(['subsr', 'series_nm'])['watched'].sum().reset_index()
df4 = df4[df4['watched']!= 0]
df4

,subsr,series_nm,watched
0,59879000,소방서 옆 경찰서,1
1,59879000,신성한 이혼,7
2,59895000,금이야 옥이야,1
3,59900000,2022 역사저널 그날,1
4,59900000,그것이알고싶다,3
...,...,...,...
2240,67140000,잠자는 숲속의 공주,1
2241,67148000,잠자는 숲속의 공주,1
2242,67148000,타요의 씽씽극장 동요2,11
2244,67161000,스파이 코드명 포춘,1


In [60]:
# 8~9울 시리즈별 시청된 총 횟수
df5 = pd.DataFrame(df11['asset_nm'].value_counts().reset_index())
df5.columns = ['series_nm', 'watched_all']
df5

,series_nm,watched_all
0,금이야 옥이야,83
1,연희공략: 건륭황제의여인,63
2,TV소설 은희,63
3,구소한야난 - 너의 온도,62
4,호심,60
...,...,...
1118,공조,1
1119,작적: 사라진 왕조의 비밀,1
1120,원샷,1
1121,유령,1


In [61]:
df6 = df4.merge(df5, on = 'series_nm', how = 'left')
df6

,subsr,series_nm,watched,watched_all
0,59879000,소방서 옆 경찰서,1,11
1,59879000,신성한 이혼,7,12
2,59895000,금이야 옥이야,1,83
3,59900000,2022 역사저널 그날,1,3
4,59900000,그것이알고싶다,3,21
...,...,...,...,...
1861,67140000,잠자는 숲속의 공주,1,1
1862,67148000,잠자는 숲속의 공주,1,1
1863,67148000,타요의 씽씽극장 동요2,11,13
1864,67161000,스파이 코드명 포춘,1,2


In [62]:
df7 = df6.groupby(['subsr'])['watched'].sum().reset_index()
df7.columns = ['subsr', 'watched_cnt']
df7

,subsr,watched_cnt
0,59879000,8
1,59895000,1
2,59900000,9
3,59921000,4
4,59930000,12
...,...,...
367,67129000,3
368,67140000,3
369,67148000,12
370,67161000,1


In [63]:
df8 = df6.merge(df7, on = 'subsr', how = 'left')
df8 = df8[df8['watched_cnt'] != 0].reset_index(drop = True)
df8

,subsr,series_nm,watched,watched_all,watched_cnt
0,59879000,소방서 옆 경찰서,1,11,8
1,59879000,신성한 이혼,7,12,8
2,59895000,금이야 옥이야,1,83,1
3,59900000,2022 역사저널 그날,1,3,9
4,59900000,그것이알고싶다,3,21,9
...,...,...,...,...,...
1861,67140000,잠자는 숲속의 공주,1,1,3
1862,67148000,잠자는 숲속의 공주,1,1,12
1863,67148000,타요의 씽씽극장 동요2,11,13,12
1864,67161000,스파이 코드명 포춘,1,2,1


In [65]:
import numpy as np
def scoring(df):
  N = df['watched_all']
  L = df['watched_cnt']
  n = df['watched']
  lam = np.log(2) / 2
  w1 = 1 - np.exp(-1 * lam * N)
  data = pd.DataFrame([N, L]).T
  def custom_weight(data):
    if data['watched_cnt'] < data['watched_all']:
      return data['watched_all'] / data['watched_cnt']
    else:
      return 1

  w2 = data.apply(custom_weight, axis = 1)

  score = (n / N) * w1 * w2

  return score

In [66]:
df8['score'] = scoring(df8)
df8

,subsr,series_nm,watched,watched_all,watched_cnt,score
0,59879000,소방서 옆 경찰서,1,11,8,0.122238
1,59879000,신성한 이혼,7,12,8,0.861328
2,59895000,금이야 옥이야,1,83,1,1.000000
3,59900000,2022 역사저널 그날,1,3,9,0.215482
4,59900000,그것이알고싶다,3,21,9,0.333103
...,...,...,...,...,...,...
1861,67140000,잠자는 숲속의 공주,1,1,3,0.292893
1862,67148000,잠자는 숲속의 공주,1,1,12,0.292893
1863,67148000,타요의 씽씽극장 동요2,11,13,12,0.906539
1864,67161000,스파이 코드명 포춘,1,2,1,0.500000


In [67]:
final_df = df8[['subsr', 'series_nm', 'score']]
final_df = final_df.merge(df3[['series_nm','ct_cl', 'genre_of_ct_cl']].drop_duplicates(), on = 'series_nm', how = 'inner')
final_df.columns = ['userid', 'program', 'score', 'main_cat', 'sub_cat']
final_df = final_df.sort_values(by = 'userid').reset_index(drop = True)
final_df

,userid,program,score,main_cat,sub_cat
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타
1,59879000,신성한 이혼,0.861328,TV드라마,기타
2,59895000,금이야 옥이야,1.000000,TV드라마,기타
3,59900000,후궁-제왕의첩,0.292893,영화,멜로
4,59900000,초대: 스와핑 데이,0.250000,영화,멜로
...,...,...,...,...,...
1878,67140000,경남 통영 2부,0.292893,우리동네,연예/오락
1879,67148000,잠자는 숲속의 공주,0.292893,키즈,기타
1880,67148000,타요의 씽씽극장 동요2,0.906539,키즈,기타
1881,67161000,스파이 코드명 포춘,0.500000,영화,액션/어드벤쳐


In [68]:
final_df

,userid,program,score,main_cat,sub_cat
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타
1,59879000,신성한 이혼,0.861328,TV드라마,기타
2,59895000,금이야 옥이야,1.000000,TV드라마,기타
3,59900000,후궁-제왕의첩,0.292893,영화,멜로
4,59900000,초대: 스와핑 데이,0.250000,영화,멜로
...,...,...,...,...,...
1878,67140000,경남 통영 2부,0.292893,우리동네,연예/오락
1879,67148000,잠자는 숲속의 공주,0.292893,키즈,기타
1880,67148000,타요의 씽씽극장 동요2,0.906539,키즈,기타
1881,67161000,스파이 코드명 포춘,0.500000,영화,액션/어드벤쳐


In [83]:
vod_name = pd.read_csv('../전처리/vod_data.csv', encoding='cp949')
vod_name.head()

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,키즈,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,키즈,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
2,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,키즈,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
3,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,키즈,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
4,지니강이 플러스 시즌7,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...


In [76]:
vod_name['Category'].unique()

array(['키즈', 'TV프로그램', '영화'], dtype=object)

In [78]:
vod_name['name'].unique()

array(['핑크퐁 동물 동요', '라바 키즈동요 Part4', '지니강이 플러스 시즌7', ..., '간질간질',
       '월간 아기상어', '놀자!'], dtype=object)

In [81]:
final_df[final_df['program'] == '놀자!']

,userid,program,score,main_cat,sub_cat
52,60169000,놀자!,0.292893,키즈,학습


In [85]:
final_df

,userid,program,score,main_cat,sub_cat
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타
1,59879000,신성한 이혼,0.861328,TV드라마,기타
2,59895000,금이야 옥이야,1.000000,TV드라마,기타
3,59900000,후궁-제왕의첩,0.292893,영화,멜로
4,59900000,초대: 스와핑 데이,0.250000,영화,멜로
...,...,...,...,...,...
1878,67140000,경남 통영 2부,0.292893,우리동네,연예/오락
1879,67148000,잠자는 숲속의 공주,0.292893,키즈,기타
1880,67148000,타요의 씽씽극장 동요2,0.906539,키즈,기타
1881,67161000,스파이 코드명 포춘,0.500000,영화,액션/어드벤쳐


In [94]:
temp = pd.merge(final_df, vod_name[['name', 'SMRY']], left_on = 'program', right_on = 'name', how = 'left')
temp = temp.drop('name', axis = 1)
temp.drop_duplicates()

,userid,program,score,main_cat,sub_cat,SMRY
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타,NaN
1,59879000,신성한 이혼,0.861328,TV드라마,기타,NaN
2,59895000,금이야 옥이야,1.000000,TV드라마,기타,NaN
3,59900000,후궁-제왕의첩,0.292893,영화,멜로,NaN
4,59900000,초대: 스와핑 데이,0.250000,영화,멜로,연극배우인 엔리케는 아내 알리시아와 거리감을 느끼며 살아가고 있다. 그러던 어느 날...
...,...,...,...,...,...,...
103165,67148000,잠자는 숲속의 공주,0.292893,키즈,기타,"[키즈스콜레]나쁜 마녀의 저주에 걸려 백년 동안 잠에 들게 된 공주, 그리고 그 공..."
103192,67148000,타요의 씽씽극장 동요2,0.906539,키즈,기타,NaN
103193,67161000,스파이 코드명 포춘,0.500000,영화,액션/어드벤쳐,"전 세계 검은 무기 거래 왕, 슈퍼 빌런 그렉. 그가 정체를 파악조차 불가능한 핸들..."
103249,67164000,밀수,0.646447,영화,액션/어드벤쳐,평화롭던 바닷가 마을 군천에 화학 공장이 들어서면서 하루아침에 일자리를 잃은 해녀들...


### 수정이 전처리 방법으로 했을 떄 (추후 바꿔야 됨)

In [102]:
# asset_nm 괄호와 괄호 안의 데이터 제거
vod89 = vod_89.copy()
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\(.*$', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace('-', '')

# n회 or n화 제거
# vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\s\d+회', '', regex=True)

# 프로그램명 제일 마지막에 . 이 찍힌 경우가 있으니 . 제거
vod89['asset_nm'] = vod89['asset_nm'].str.rstrip('.')

# use_tms 분 단위로 변경
vod89['use_tms'] = round(vod89['use_tms'] / 60, 1)

# disp_rtm 문자열을 분 단위로 변경
def convert_runtime(runtime_str):
    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

vod89['disp_rtm'] = vod89['disp_rtm'].apply(convert_runtime)

vod89

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737
...,...,...,...,...,...,...,...,...,...
10652,60169000,간질간질,키즈,학습,6.0,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,6,20230914211110
10653,66317000,알사탕,키즈,학습,8.0,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,8,20230903223544
10654,66317000,알사탕,키즈,학습,3.3,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,8,20230903223210
10655,66900000,월간 아기상어 01회,키즈,학습,0.8,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,1,20230904120057


In [106]:
vod89 = vod89.dropna()
vod89.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737


In [107]:
# 프로그램 회차별 정보
df1 = vod89[['asset_nm', 'ct_cl', 'genre_of_ct_cl', 'ACTR_DISP', 'disp_rtm']].drop_duplicates().reset_index(drop = True)
df1.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78


In [108]:
# 유저마다 각 회차를 본 시간을 더함
df2 = pd.DataFrame(vod89.groupby(['subsr','asset_nm'])['use_tms'].sum()).reset_index()
df2.head()

,subsr,asset_nm,use_tms
0,59879000,소방서 옆 경찰서 05회,2.4
1,59879000,소방서 옆 경찰서 06회,22.1
2,59879000,신성한 이혼 01회,12.6
3,59879000,신성한 이혼 02회,16.8
4,59879000,신성한 이혼 03회,4.3


In [109]:
# 회차별 정보와 유저 시청정보를 합친 데이터프레임
df3 = df2.merge(df1, on = 'asset_nm', how = 'left')
df3 = df3[(df3['disp_rtm'] != 0) & (df3['use_tms'] != 0)]
df3.head()

,subsr,asset_nm,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,59879000,소방서 옆 경찰서 05회,2.4,TV드라마,기타,"김래원,손호준,공승연",69
1,59879000,소방서 옆 경찰서 06회,22.1,TV드라마,기타,"김래원,손호준,공승연",59
2,59879000,신성한 이혼 01회,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64
3,59879000,신성한 이혼 02회,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62
4,59879000,신성한 이혼 03회,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63


In [110]:
# 시청시간으로 유저가 이 회차를 시청했는지 여부를 결정
## 시청시간을 런타임으로 나눠 0.1 이 넘으면 시청한 것으로 간주
df3['watched'] = df3['use_tms'] / df3['disp_rtm']
df3['watched'] = df3['watched'].apply(lambda x : 1 if x >= 0.1 else 0)
df3.head()

,subsr,asset_nm,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,watched
0,59879000,소방서 옆 경찰서 05회,2.4,TV드라마,기타,"김래원,손호준,공승연",69,0
1,59879000,소방서 옆 경찰서 06회,22.1,TV드라마,기타,"김래원,손호준,공승연",59,1
2,59879000,신성한 이혼 01회,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,1
3,59879000,신성한 이혼 02회,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62,1
4,59879000,신성한 이혼 03회,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63,0


In [111]:
# 회차 제거
df11 = df1.copy()

df11['asset_nm'] = df11['asset_nm'].str.replace(r'\s\d+회', '', regex=True)
df11.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",78


In [112]:
# 유저 시청 정보의 회차 제거한 변수 추가
df3['series_nm'] = df3['asset_nm'].str.replace(r'\s\d+회', '', regex=True)
df3.head()

,subsr,asset_nm,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,2.4,TV드라마,기타,"김래원,손호준,공승연",69,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,22.1,TV드라마,기타,"김래원,손호준,공승연",59,1,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,1,신성한 이혼
3,59879000,신성한 이혼 02회,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62,1,신성한 이혼
4,59879000,신성한 이혼 03회,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63,0,신성한 이혼


In [113]:
df33 = df3[['subsr', 'series_nm', 'ct_cl',	'genre_of_ct_cl', 'ACTR_DISP',	'watched']].copy()
df33.head()

,subsr,series_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,watched
0,59879000,소방서 옆 경찰서,TV드라마,기타,"김래원,손호준,공승연",0
1,59879000,소방서 옆 경찰서,TV드라마,기타,"김래원,손호준,공승연",1
2,59879000,신성한 이혼,TV드라마,기타,"조승우,한혜진,김성균,정문성",1
3,59879000,신성한 이혼,TV드라마,기타,"조승우,한혜진,김성균,정문성",1
4,59879000,신성한 이혼,TV드라마,기타,"조승우,한혜진,김성균,정문성",0


In [114]:
# 유저의 시리즈별 시청 횟수
df4 = df33.groupby(['subsr', 'series_nm'])['watched'].sum().reset_index()
df4 = df4[df4['watched']!= 0]
df4

,subsr,series_nm,watched
0,59879000,소방서 옆 경찰서,1
1,59879000,신성한 이혼,7
2,59895000,금이야 옥이야,1
3,59900000,2022 역사저널 그날,1
4,59900000,그것이알고싶다,3
...,...,...,...
2240,67140000,잠자는 숲속의 공주,1
2241,67148000,잠자는 숲속의 공주,1
2242,67148000,타요의 씽씽극장 동요2,11
2244,67161000,스파이 코드명 포춘,1


In [115]:
# 8~9울 시리즈별 시청된 총 횟수
df5 = pd.DataFrame(df11['asset_nm'].value_counts().reset_index())
df5.columns = ['series_nm', 'watched_all']
df5

,series_nm,watched_all
0,금이야 옥이야,83
1,연희공략: 건륭황제의여인,63
2,TV소설 은희,63
3,인간극장,55
4,런닝맨,54
...,...,...
1118,공조,1
1119,작적: 사라진 왕조의 비밀,1
1120,원샷,1
1121,유령,1


In [116]:
df6 = df4.merge(df5, on = 'series_nm', how = 'left')
df6

,subsr,series_nm,watched,watched_all
0,59879000,소방서 옆 경찰서,1,11
1,59879000,신성한 이혼,7,12
2,59895000,금이야 옥이야,1,83
3,59900000,2022 역사저널 그날,1,3
4,59900000,그것이알고싶다,3,21
...,...,...,...,...
1861,67140000,잠자는 숲속의 공주,1,1
1862,67148000,잠자는 숲속의 공주,1,1
1863,67148000,타요의 씽씽극장 동요2,11,13
1864,67161000,스파이 코드명 포춘,1,2


In [117]:
df7 = df6.groupby(['subsr'])['watched'].sum().reset_index()
df7.columns = ['subsr', 'watched_cnt']
df7

,subsr,watched_cnt
0,59879000,8
1,59895000,1
2,59900000,9
3,59921000,4
4,59930000,12
...,...,...
367,67129000,3
368,67140000,3
369,67148000,12
370,67161000,1


In [118]:
df8 = df6.merge(df7, on = 'subsr', how = 'left')
df8 = df8[df8['watched_cnt'] != 0].reset_index(drop = True)
df8

,subsr,series_nm,watched,watched_all,watched_cnt
0,59879000,소방서 옆 경찰서,1,11,8
1,59879000,신성한 이혼,7,12,8
2,59895000,금이야 옥이야,1,83,1
3,59900000,2022 역사저널 그날,1,3,9
4,59900000,그것이알고싶다,3,21,9
...,...,...,...,...,...
1861,67140000,잠자는 숲속의 공주,1,1,3
1862,67148000,잠자는 숲속의 공주,1,1,12
1863,67148000,타요의 씽씽극장 동요2,11,13,12
1864,67161000,스파이 코드명 포춘,1,2,1


In [119]:
import numpy as np
def scoring(df):
  N = df['watched_all']
  L = df['watched_cnt']
  n = df['watched']
  lam = np.log(2) / 2
  w1 = 1 - np.exp(-1 * lam * N)
  data = pd.DataFrame([N, L]).T
  def custom_weight(data):
    if data['watched_cnt'] < data['watched_all']:
      return data['watched_all'] / data['watched_cnt']
    else:
      return 1

  w2 = data.apply(custom_weight, axis = 1)

  score = (n / N) * w1 * w2

  return score

In [120]:
df8['score'] = scoring(df8)
df8

,subsr,series_nm,watched,watched_all,watched_cnt,score
0,59879000,소방서 옆 경찰서,1,11,8,0.122238
1,59879000,신성한 이혼,7,12,8,0.861328
2,59895000,금이야 옥이야,1,83,1,1.000000
3,59900000,2022 역사저널 그날,1,3,9,0.215482
4,59900000,그것이알고싶다,3,21,9,0.333103
...,...,...,...,...,...,...
1861,67140000,잠자는 숲속의 공주,1,1,3,0.292893
1862,67148000,잠자는 숲속의 공주,1,1,12,0.292893
1863,67148000,타요의 씽씽극장 동요2,11,13,12,0.906539
1864,67161000,스파이 코드명 포춘,1,2,1,0.500000


In [128]:
final_df = df8[['subsr', 'series_nm', 'score']]
final_df = final_df.merge(df3[['series_nm','ct_cl', 'genre_of_ct_cl']].drop_duplicates(), on = 'series_nm', how = 'inner')
final_df.columns = ['userid', 'program', 'score', 'main_cat', 'sub_cat']
final_df = final_df.sort_values(by = 'userid').reset_index(drop = True)
final_df

,userid,program,score,main_cat,sub_cat
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타
1,59879000,신성한 이혼,0.861328,TV드라마,기타
2,59895000,금이야 옥이야,1.000000,TV드라마,기타
3,59900000,후궁제왕의첩,0.292893,영화,멜로
4,59900000,초대: 스와핑 데이,0.250000,영화,멜로
...,...,...,...,...,...
1878,67140000,경남 통영 2부,0.292893,우리동네,연예/오락
1879,67148000,잠자는 숲속의 공주,0.292893,키즈,기타
1880,67148000,타요의 씽씽극장 동요2,0.906539,키즈,기타
1881,67161000,스파이 코드명 포춘,0.500000,영화,액션/어드벤쳐


In [129]:
final_df['program'] = final_df['program'].str.replace('-', ' ')

In [126]:
vod_smry = pd.read_csv('../전처리/vod_data_addSummary.csv', index_col=0)
vod_smry = vod_smry[['name', 'Summary']]

,name,Summary
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...
1,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...
2,지니강이 플러스 시즌9,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.
3,헤이지니 럭키강이 Part1,지니가 초대형 볼풀장에 놀러간 이야기.
4,우당탕탕 에디의 실험실,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...
...,...,...
3633,엄마가 화났다,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...
3634,핑크퐁 자동차 동화,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...
3635,간질간질,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
3636,월간 아기상어,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...


In [130]:
final_df

,userid,program,score,main_cat,sub_cat
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타
1,59879000,신성한 이혼,0.861328,TV드라마,기타
2,59895000,금이야 옥이야,1.000000,TV드라마,기타
3,59900000,후궁제왕의첩,0.292893,영화,멜로
4,59900000,초대: 스와핑 데이,0.250000,영화,멜로
...,...,...,...,...,...
1878,67140000,경남 통영 2부,0.292893,우리동네,연예/오락
1879,67148000,잠자는 숲속의 공주,0.292893,키즈,기타
1880,67148000,타요의 씽씽극장 동요2,0.906539,키즈,기타
1881,67161000,스파이 코드명 포춘,0.500000,영화,액션/어드벤쳐


In [135]:
pd.merge(final_df, vod_smry[['name', 'Summary']], left_on = 'program', right_on='name', how = 'left').drop_duplicates()

,userid,program,score,main_cat,sub_cat,name,Summary
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타,NaN,NaN
1,59879000,신성한 이혼,0.861328,TV드라마,기타,신성한 이혼,"이혼 전문 변호사의 파란만장한 이야기를 그린 작품으로, '이혼'이라는 삶의 험난한 ..."
12,59895000,금이야 옥이야,1.000000,TV드라마,기타,금이야 옥이야,싱글대디 금강산과 입양아 옥미래를 중심으로 금이야 옥이야 자식을 키우며 성장해가는 ...
33,59900000,후궁제왕의첩,0.292893,영화,멜로,NaN,NaN
34,59900000,초대: 스와핑 데이,0.250000,영화,멜로,NaN,NaN
...,...,...,...,...,...,...,...
4111,67140000,경남 통영 2부,0.292893,우리동네,연예/오락,경남 통영 2부,파크골프동호회리그 시니어부 3R이모저모 ep1 영상스케치
4112,67148000,잠자는 숲속의 공주,0.292893,키즈,기타,NaN,NaN
4113,67148000,타요의 씽씽극장 동요2,0.906539,키즈,기타,NaN,NaN
4114,67161000,스파이 코드명 포춘,0.500000,영화,액션/어드벤쳐,NaN,NaN
